In [2]:
from transformers import AutoModelForSeq2SeqLM , AutoTokenizer

In [3]:
model_name='google/flan-t5-small'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [4]:
from datasets import load_dataset,Dataset

In [5]:
data=load_dataset("databricks/databricks-dolly-15k")
data

README.md: 0.00B [00:00, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

In [6]:
data['train'][1]

{'instruction': 'Which is a species of fish? Tope or Rope',
 'context': '',
 'response': 'Tope',
 'category': 'classification'}

In [7]:
def data_preprocessing(train):
  example=(train['instruction']+'\n'+train['context'])
  data_ids=tokenizer(example,padding='max_length',truncation=True,max_length=256,return_tensors='pt').input_ids[0]
  output_ids=tokenizer(train['response'],padding='max_length',truncation=True,max_length=256,return_tensors='pt').input_ids[0]

  return{'input_ids':data_ids,'labels':output_ids}


In [18]:
data_for_tokenization_sample=data['train'].shuffle(seed=42).select(range(2000))

In [19]:
tokenized_data=data_for_tokenization_sample.map(data_preprocessing)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [20]:
from transformers import TrainingArguments,Trainer

In [21]:
training_args=TrainingArguments(
    output_dir='/outputs',
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_strategy='epoch',
    logging_steps=10,
    disable_tqdm=False,
    report_to='wandb',
    label_names=['labels']
)

In [22]:
from peft import LoraConfig,get_peft_model,TaskType
lora=LoraConfig(
    r=8,
    lora_alpha=16,lora_dropout=.1,
    bias='none',
    task_type=TaskType.SEQ_2_SEQ_LM

)

In [23]:
model1=get_peft_model(model,lora)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [24]:
trainer1=Trainer(
    model=model1,
    args=training_args,
    train_dataset=tokenized_data
)

In [25]:
trainer1.train()

Step,Training Loss
10,11.362749
20,11.310529
30,11.340077
40,11.334685
50,11.229969
60,11.275082
70,11.248275
80,11.146552
90,11.060597
100,11.081218


TrainOutput(global_step=750, training_loss=10.487282908121745, metrics={'train_runtime': 232.1663, 'train_samples_per_second': 25.844, 'train_steps_per_second': 3.23, 'total_flos': 560842407936000.0, 'train_loss': 10.487282908121745, 'epoch': 3.0})

In [27]:
from peft import IA3Config

In [34]:
ia3=IA3Config(task_type=TaskType.SEQ_2_SEQ_LM,
              target_modules=['q','v','wi','wo'])
model2=get_peft_model(model,ia3)

trainer2=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [35]:
trainer2.train()

Step,Training Loss
10,9.750645
20,9.789282
30,9.752444
40,9.787213
50,9.793114
60,9.767111
70,9.703996
80,9.708582
90,9.709162
100,9.739724


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=750, training_loss=9.646906514485677, metrics={'train_runtime': 301.638, 'train_samples_per_second': 19.891, 'train_steps_per_second': 2.486, 'total_flos': 561163272192000.0, 'train_loss': 9.646906514485677, 'epoch': 3.0})